In [1]:
import pandas as pd
import numpy as np
import math
import datetime

table=pd.read_csv("../Dati/orari/route.csv", delimiter=";")
time_table=pd.read_csv("../Dati/orari/route_time.csv", delimiter=";")
dft = pd.read_csv("../Dati/distanze/times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";")
fur2 = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";")

def get_row(df, string):
    return pd.Series(df.index).str.contains(string, case=False)

def get_column(df, string):
    return pd.Series(df.columns).str.contains(string, case=False)

def difference(day, index, index2):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index2], '%H:%M')
    return ((time2-time).seconds)/60


In [2]:
def new_route(schedule):
    day='lunedi'
    times = []
    keys = []
    row = dft[get_row(dft, 'Cremona, sesto 39').values]
    key1 = ''
    key2 = ''
    impossibile = []
    for i in range(1,len(schedule)-1):
        key1 = row.index[0]
        try:
            paese=schedule[i]
        except:
            continue
        #Distanze in tempo dal paese corrente di ognuno dei furgoni alla destinazione
        value=row.loc[:,get_column(dft, paese).values].values[0][0]
        times.append(round(value))
        row = dft[get_row(dft, paese).values]
        key2 = row.index[0]
        keys.append(key1 + ' - ' + key2)
    
    steps_times1 = pd.Series(times , keys)
    #creo una copia della serie
    #steps_times2 = steps_times1.copy()
    #creo una copia dei tempi
    times1 = times[:]
    #times2 = times[:]

    #aggiungo la chiave per l'ultima destinazione considerata e la casa dell'autista

    #autista 1
    row = dft[get_row(dft, paese).values]
    value=row.loc[:,get_column(dft, 'Ostiano, Mazzini, 1').values].values[0][0]
    s1 = pd.Series(round(value), [row.index[0] + ' - Ostiano, Mazzini, 1'])
    steps_times1 = steps_times1.append(s1)
    times1.append(round(value))

    """#autista 2
    value=row.loc[:,get_column(dft, 'San Giovanni in Croce, Feudatari, 25 A').values].values[0][0]
    s2 = pd.Series(round(value), [row.index[0] + ' - San Giovanni in Croce, Feudatari, 25 A'])
    steps_times2 = steps_times2.append(s2)
    times2.append(round(value))

    if(sum(times1) < sum(times2)):"""
    return steps_times1
    """else:
        return steps_times2"""

In [3]:
#first attempt to local search algorithm
def find_best_position(steps_times, to_insert):
    times = []
    key2 = []
    for i in range(len(steps_times)):
        cities = steps_times.keys()[i].split('-')
        row1 = dft[get_row(dft, cities[0].strip()).values]
        #time from city 1 to current (to insert)
        f_c1_to_curr = row1.loc[:,get_column(dft, to_insert).values].values[0][0]
        row2 = dft[get_row(dft, to_insert).values]
        #time from current (to insert) to city 2
        f_curr_to_c2 = row2.loc[:,get_column(dft, cities[1].strip()).values].values[0][0]
        times.append(f_c1_to_curr + f_curr_to_c2)
    s3 = pd.Series(times, steps_times.keys())
    return s3

In [10]:
#furgoncino 1 (MON, FRI)
fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";")

schedule = []
schedule.append('Cremona, Sesto 39'.lower())
for i in range(len(fur1['lunedi'])):
    schedule.append(dft[get_row(dft, fur1['lunedi'][i]).values].index[0].lower())
    
schedule.append('Ostiano, Mazzini, 1'.lower())
#schedule.append('San Giovanni in Croce, Feudatari, 25 A'.lower())

print(schedule)
l = len(schedule)-1
moved = []
i = 1
#finché in moved non ci sono tutte le città (meno Cremona e Ostiano)
while len(moved) < l-1:
    #faccio il pop della città
    to_insert = schedule.pop(i)
    #controllo se l'ho già considerata
    if(to_insert not in moved):
        #(se no) la inserisco nella lista di quelle già considerate
        moved.append(to_insert)
        #calcolo la nuova route tra le città rimanenti
        route = new_route(schedule)
        #trovo la posizione migliore
        best = find_best_position(route, to_insert).idxmin()
        city = best.split('-')[0].strip().lower()
        try:
            #trovo l'indice della posizione migliore
            position = schedule.index(city) + 1
        except:
            #problema nome diverso
            if(city == 'casale cremasco'):
                city = 'casale cremasco-vidolasco, roma, 56'
                position = schedule.index(city) + 1
        #inserisco la città nella posizione corretta
        schedule.insert(position ,to_insert)
    else:
        #(se si) reinserisco la città nella sua posizione e vado avanti
        schedule.insert(i ,to_insert)
        #incremento l'indice per la prossima città
        i += 1

['cremona, sesto 39', 'sesto ed uniti, roma, 4', 'grumello cremonese ed uniti, roma, 2', 'soresina, giacomo matteotti, 6', 'trigolo, roma, 26', 'castelleone, roma, 67', 'montodine,  xxv aprile, 10', 'ripalta cremasca, roma, 5', 'ricengo, guglielmo marconi', 'sergnano, iv novembre,  10', 'casale cremasco-vidolasco, roma, 56', 'camisano, trieste, 57', 'capralba, piave, 2', 'soncino, dante alighieri, 23', 'pizzighettone, giuseppe garibaldi, 18', 'ostiano, mazzini, 1']


In [11]:
schedule

['cremona, sesto 39',
 'sesto ed uniti, roma, 4',
 'pizzighettone, giuseppe garibaldi, 18',
 'grumello cremonese ed uniti, roma, 2',
 'soresina, giacomo matteotti, 6',
 'soncino, dante alighieri, 23',
 'trigolo, roma, 26',
 'castelleone, roma, 67',
 'ripalta cremasca, roma, 5',
 'montodine,  xxv aprile, 10',
 'ricengo, guglielmo marconi',
 'capralba, piave, 2',
 'sergnano, iv novembre,  10',
 'casale cremasco-vidolasco, roma, 56',
 'camisano, trieste, 57',
 'ostiano, mazzini, 1']